## TOM, 

I got it to work. The first 5 cells are needed and then everything after Combined Systems. Naturally you can remove everything used to view the results

In [7]:
import pandas as pd
import numpy as np

In [10]:
#load movies and ratings

movies = pd.read_csv('Data/movies_sml.csv')
ratings = pd.read_hdf('Data/ratings_hdf.h5')

In [11]:
# load movies with bag of words. Needed for content filtering

movies_bow = pd.read_csv('Data/movies_bow.csv')

In [60]:
# #load ratings matrix.Needed for Collaborative filtering
# ratings_matrix = pd.read_hdf('Data/ratings_mx.h5')

In [12]:
# load predictions
preds = pd.read_hdf('Data/predsfin_hdf.h5')

In [13]:
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175991,175993,175995,175999,176001,176003,176007,176157,176211,176271
0,2.560615,0.031274,0.000000,0.02062,0.000000,0.264343,0.000000,0.000000,0.000000,0.201482,...,0.001771,0.001789,0.001807,0.001807,0.001807,0.001824,0.001754,0.001644,0.002967,0.002237
1,0.441066,0.000000,0.211197,0.00000,0.153935,2.553422,0.270768,0.000000,0.102689,0.535472,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.06451,0.000000,0.064820,0.000000,0.012888,0.000000,0.000000,...,0.007978,0.007942,0.007906,0.007906,0.007906,0.007870,0.008014,0.009173,0.005091,0.006427
3,1.296987,1.336419,0.000000,0.00000,0.000000,3.126836,0.000000,0.000000,0.094546,1.727142,...,0.000000,0.000000,0.000056,0.000056,0.000056,0.000154,0.000000,0.002996,0.000401,0.000000
4,4.743518,0.725185,0.000000,0.00000,0.097943,0.340626,0.199099,0.000000,0.000000,1.098070,...,0.001233,0.001167,0.001102,0.001102,0.001102,0.001036,0.001298,0.000000,0.001855,0.002679


###### test collab system first to see if any additional files are needed

In [14]:


def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userId - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

#     print (f"User {0} has already rated {1} movies. format(userID, user_full.shape[0]")
#     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    rec_movies = recommendations['Title']
    already_rated = user_full['Title_x']
    return  already_rated, rec_movies

In [15]:
already_rated, predictions = recommend_movies(preds, 60, movies, ratings, 20)

In [16]:
predictions

6447                                            Inception
1698                                           Fight Club
6150                                               Avatar
1946                                            Gladiator
3305                                    Kill Bill: Vol. 1
6231                                       Shutter Island
5856                                 Inglourious Basterds
2274                                                Shrek
3154    Pirates of the Caribbean: The Curse of the Bla...
3549                                    Kill Bill: Vol. 2
2229                                              Memento
5656                                  Slumdog Millionaire
2505                                       Ocean's Eleven
1106                                    Good Will Hunting
5898                                                   Up
5457                                             Iron Man
7585                                     Django Unchained
6155          

###### test content system first to see if any additional files are needed

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# Initializing and generating the count matrix 

count = CountVectorizer()
count_matrix = count.fit_transform(movies_bow['bag_of_words'])

In [19]:
# Generating the cosine similarity matrix

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [20]:
#map index to Title. This will be used in the function and is integral to 
# how we access the recommended movies

indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

#Create recommendation function

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:26]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_bow['Title'].iloc[movie_indices]

In [21]:
get_recommendations('Rumble in the Bronx')

10841                   Kung Fu Yoga
4038                       Who Am I?
3401                     Crime Story
660                       Supercop 2
679      Around the World in 80 Days
680      Around the World in 80 Days
1127                       Mr. Magoo
1393                       Rush Hour
1565                      Black Mask
1970                   Shanghai Noon
2391                     Rush Hour 2
3983                     City Hunter
4852                     Thunderbolt
5184                     Rush Hour 3
9400                    Dragon Blade
10585                      Skiptrace
10678                Railroad Tigers
1209                    Mr. Nice Guy
2993                Shanghai Knights
4294                        Gorgeous
4536                New Police Story
6246               Shinjuku Incident
6658              Little Big Soldier
4850                    Swordsman II
0                          Toy Story
Name: Title, dtype: object

#### Combined Systems

In [22]:
#Content-based Recommendations Sections
indices = pd.Series(movies_bow.index, index = movies_bow['Title'])

indices

Title
Toy Story                                                       0
Jumanji                                                         1
Grumpier Old Men                                                2
Waiting to Exhale                                               3
Father of the Bride Part II                                     4
Heat                                                            5
Heat                                                            6
Sabrina                                                         7
Sabrina                                                         8
Tom and Huck                                                    9
Sudden Death                                                   10
GoldenEye                                                      11
Dracula: Dead and Loving It                                    12
Balto                                                          13
Nixon                                                          14
Cutt

In [23]:
idx = indices['American History X']

In [24]:
sim_scores = list(enumerate(cosine_sim[idx]))


In [25]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [26]:
sim_scores = sim_scores[1:26]

In [27]:
movie_indices = [i[0] for i in sim_scores]

In [28]:
content_results = pd.DataFrame(movies_bow['movieId'].iloc[movie_indices])


content_results 

,movieId
396,487
935,1461
4250,31045
4252,31083
1929,3536
9581,137900
104,113
228,268
467,589
741,1057


In [34]:
# Collaborative Filtering Sections

# get user predictions

user= 60 # User ID starts at 1, not 0

sorted_user_predictions = pd.DataFrame(preds.iloc[user].sort_values(ascending=False).reset_index())

sorted_user_predictions.head()

,movieId,60
0,6942,3.180631
1,56367,3.137189
2,597,3.079730
3,7361,2.900276
4,2572,2.807026


In [35]:
# Get the movies the user originally rated
user_data = ratings[ratings.userId == user]

#combine the movies the user originally rated with the movies dataset and only keeping the movies
#the user rated

user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))

In [36]:
print(user_full.columns)

user_full = user_full[['userId', 'movieId', 'Title_x', 'rating', 'genres', 'Actors', 'Director', 'Plot', 'Poster']].\
            rename(columns = {'Title_x': 'Title'})


user_full.head()

Index(['userId', 'num_user_rated', 'Title_x', 'rating', 'movieId', 'Title_y',
       'genres', 'Actors', 'Director', 'Plot', 'Poster'],
      dtype='object')


,userId,movieId,Title,rating,genres,Actors,Director,Plot,Poster


In [32]:

num_recommendations = 1000

I noticed if the user doesn't watch anything similar to the movie inputted, we need to return a wide range of values from the sorted ratings. This is why num_recommendations is so high. 

One workaround is to limit num_recommendations to 100 and if there are no matches between content and collaborative results, set a message saying you will not like movies like 'movie name'

In [37]:
# Original Code from Collaborative Filter Function

# Recommend the highest predicted rating movies that the user hasn't seen yet.
movie_preds = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations]
)




In [38]:
movie_preds.shape

(1000, 9)

In [39]:
movie_preds.shape

(1000, 9)

In [40]:
# lookup movie predictions from content system from this dataframe

movie_recs = pd.merge(content_results, movie_preds, how='left', on='movieId').\
    sort_values('Predictions', ascending=False).dropna().head()

In [41]:
movie_recs

,movieId,Title,genres,Actors,Director,Plot,Poster,index,Predictions
13,2959,Fight Club,Action|Crime|Drama|Thriller,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier",David Fincher,An insomniac office worker and a devil-may-car...,https://m.media-amazon.com/images/M/MV5BMjJmYT...,67.0,0.980892
4,3536,Keeping the Faith,Comedy|Drama|Romance,"Ben Stiller, Edward Norton, Jenna Elfman, Anne...",Edward Norton,"Two friends, a priest and a rabbi, fall in lov...",https://m.media-amazon.com/images/M/MV5BMTMxNz...,327.0,0.249671
12,2772,Detroit Rock City,Comedy,"Giuseppe Andrews, James DeBello, Edward Furlon...",Adam Rifkin,"In 1978, four rebellious teenagers try to scam...",https://m.media-amazon.com/images/M/MV5BMmU5ZT...,948.0,0.071727
